In [12]:
import sqlite3
import pandas as pd

In [3]:
connection = sqlite3.connect("data/sql-murder-mystery.db")
crsr = connection.cursor()

In [4]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

In [8]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

crime_scene_report
drivers_license
person
facebook_event_checkin
interview
get_fit_now_member
get_fit_now_check_in
income
solution


In [18]:
query = '''
SELECT sql
FROM sqlite_master
WHERE name = 'crime_scene_report'
'''
crime_scene =  sql_query(query)
crime_scene

,sql
0,CREATE TABLE crime_scene_report (\n dat...


In [27]:
query = '''
SELECT description 
FROM crime_scene_report
WHERE type = "murder"
AND date = "20180115"
AND city = "SQL City"
'''
crime_scene = sql_query(query)
crime_scene

<bound method NDFrame._add_numeric_operations.<locals>.all of 0    Security footage shows that there were 2 witne...
Name: description, dtype: object>

Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".

In [30]:
query = '''
SELECT * 
FROM person
WHERE address_street_name = "Northwestern Dr"
ORDER BY address_number DESC
LIMIT 5
'''
person = sql_query(query)
person

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
1,17729,Lasonya Wildey,439686,3824,Northwestern Dr,917817122
2,53890,Sophie Tiberio,957671,3755,Northwestern Dr,442830147
3,73368,Torie Thalmann,773862,3697,Northwestern Dr,341559436
4,96595,Coretta Cubie,303645,3631,Northwestern Dr,378403829


In [36]:
query = '''
SELECT * 
FROM person
WHERE address_street_name = "Franklin Ave"
AND name LIKE "Annabel%"
'''
person2 = sql_query(query)
person2

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [44]:
query = '''
SELECT * 
FROM get_fit_now_member
WHERE person_id = "16371"
OR person_id = "14887"
'''
person_id = sql_query(query)
person_id

,id,person_id,name,membership_start_date,membership_status
0,90081,16371,Annabel Miller,20160208,gold


In [51]:
query = '''
SELECT * 
FROM interview
WHERE (person_id = "16371" OR person_id = "14887")
'''
person_id = sql_query(query)
person_id

,person_id,transcript
0,14887,I heard a gunshot and then saw a man run out. ...
1,16371,"I saw the murder happen, and I recognized the ..."


- 14887
I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".

- 16371
I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.

In [52]:
query = '''
SELECT * 
FROM get_fit_now_check_in
WHERE check_in_date = "20180109"
AND membership_id LIKE "48Z%"
'''
gym_access = sql_query(query)
gym_access

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


In [53]:
query = '''
SELECT * 
FROM get_fit_now_member
WHERE membership_status = "gold"
'''
gym_access = sql_query(query)
gym_access

,id,person_id,name,membership_start_date,membership_status
0,NL318,65076,Everette Koepke,20170926,gold
1,0YJ24,80651,Waneta Wellard,20171206,gold
2,5Y28Y,15218,Millicent Yessios,20180205,gold
3,A5N3S,24541,Mary Cameron,20170501,gold
4,R4J4S,13703,Lien Yasin,20170414,gold
...,...,...,...,...,...
63,0T622,98782,Sarina Overpeck,20171109,gold
64,BL3KC,24033,Collene Gulde,20170913,gold
65,48Z7A,28819,Joe Germuska,20160305,gold
66,48Z55,67318,Jeremy Bowers,20160101,gold


In [55]:
query = '''
SELECT * 
FROM drivers_license
WHERE plate_number LIKE "%H42W%"
AND gender = "male"
'''
car_plate = sql_query(query)
car_plate

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [62]:
query = '''
SELECT *
FROM drivers_license 
JOIN person
ON drivers_license.id = person.license_id
WHERE gender = "male"
AND plate_number LIKE "%H42W%"
'''
driver_join = sql_query(query)
driver_join

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id,name,license_id,address_number,address_street_name,ssn
0,664760,21,71,black,black,male,4H42WR,Nissan,Altima,51739,Tushar Chandra,664760,312,Phi St,137882671
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [68]:
query = '''
SELECT *
FROM get_fit_now_member 
JOIN person
ON get_fit_now_member.person_id = person.id
WHERE membership_status = "gold"
AND (person_id = "51739" OR person_id = "67318")
'''
gym_join = sql_query(query)
gym_join

,id,person_id,name,membership_start_date,membership_status,id,name,license_id,address_number,address_street_name,ssn
0,48Z55,67318,Jeremy Bowers,20160101,gold,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [69]:
query = '''
SELECT * 
FROM interview
WHERE (person_id = "67318")
'''
person_id = sql_query(query)
person_id

,person_id,transcript
0,67318,I was hired by a woman with a lot of money. I ...


 I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.

In [77]:
query = '''
SELECT * 
FROM drivers_license
JOIN person
ON person.license_id = drivers_license.id
WHERE gender = "female"
AND car_make = "Tesla"
AND car_model = "Model S"
AND hair_color = "red"
AND (height >= 65 AND height <= 67)
'''
car_join = sql_query(query)
car_join

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id,name,license_id,address_number,address_street_name,ssn
0,918773,48,65,black,red,female,917UU3,Tesla,Model S,78881,Red Korb,918773,107,Camerata Dr,961388910
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S,90700,Regina George,291182,332,Maple Ave,337169072
2,202298,68,66,green,red,female,500123,Tesla,Model S,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [93]:
query = '''
SELECT *
FROM facebook_event_checkin
JOIN person
ON person.id = facebook_event_checkin.person_id
WHERE event_name LIKE "%SQL%"
AND date LIKE "%201712%"
AND (name LIKE "Red%" OR name LIKE "Regina%" OR name LIKE "Miranda%")
'''

facebook_join = sql_query(query)
facebook_join

,person_id,event_id,event_name,date,id,name,license_id,address_number,address_street_name,ssn
0,99716,1143,SQL Symphony Concert,20171206,99716,Miranda Priestly,202298,1883,Golden Ave,987756388
1,99716,1143,SQL Symphony Concert,20171212,99716,Miranda Priestly,202298,1883,Golden Ave,987756388
2,99716,1143,SQL Symphony Concert,20171229,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [95]:
query = '''
SELECT * 
FROM income
JOIN person
ON person.ssn = income.ssn
WHERE name LIKE "Miranda%"

'''

income_join = sql_query(query)
income_join

,ssn,annual_income,id,name,license_id,address_number,address_street_name,ssn
0,987756388,310000,99716,Miranda Priestly,202298,1883,Golden Ave,987756388
